In [1]:
import utils
import skimage
import skimage.morphology
import numpy as np

In [2]:
def remove_noise(im: np.ndarray) -> np.ndarray:
    """
        A function that removes noise in the input image.
        args:
            im: np.ndarray of shape (H, W) with boolean values (dtype=np.bool)
        return:
            (np.ndarray) of shape (H, W). dtype=np.bool
    """
    # Found different structuring elements in scikit-image's own api documentation
    # https://scikit-image.org/docs/dev/auto_examples/numpy_operations/plot_structuring_elements.html#sphx-glr-auto-examples-numpy-operations-plot-structuring-elements-py
    structuring_element = skimage.morphology.octagon(6,4) # Close, but removes a little bit of the tips of triangle
    structuring_element = skimage.morphology.disk(7) # Best results
    # disk with value 7 seems to be giving the best results. If value is smaller the result is more noisy, and larger value seems to break the triangle structure

    # As we both want to remove the noise around the triangle, and fill the holes in it, we should both apply opening and closing
    # Skimage seems to use 'selem' instead of 'footprint' in this version
    skimage.morphology.binary_opening(im, selem=structuring_element, out=im)
    skimage.morphology.binary_closing(im, selem=structuring_element, out=im)
    return im

In [3]:
if __name__ == "__main__":
    im = utils.read_image("noisy.png")
    binary_image = (im != 0)
    noise_free_image = remove_noise(binary_image)

    assert im.shape == noise_free_image.shape, "Expected image shape ({}) to be same as resulting image shape ({})".format(
            im.shape, noise_free_image.shape)
    assert noise_free_image.dtype == bool, "Expected resulting image dtype to be np.bool. Was: {}".format(
            noise_free_image.dtype)

    noise_free_image = utils.to_uint8(noise_free_image)
    utils.save_im("noisy-filtered.png", noise_free_image)

Reading image: img/noisy.png
Saving image to: img/processed/noisy-filtered.png
